In [ ]:
import os
import json
import pandas as pd
from PIL import Image
import numpy as np

In [ ]:
# Load metadata file
meta_path = 'PKLot/train/_annotations.coco.json'
with open(meta_path) as file:
    meta_train = json.load(file)


In [ ]:
# Create DataFrame from the metadata
train = pd.DataFrame(meta_train['images'])
df_meta = pd.DataFrame(meta_train['annotations'])
train = pd.DataFrame.merge(train,df_meta, how='left', left_on='id', right_on='image_id')
train = train.dropna()
train['id_y'] = train['id_y'].values.astype(int)
train['category_id'] = train['category_id'].values.astype(int)
train['area'] = train['area'].values.astype(int)
train['iscrowd'] = train['iscrowd'].values.astype(int)

In [ ]:
train_img_folder = 'PKLot/train'
image = []
for img in train['file_name'].unique():
    img_path = os.path.join(train_img_folder, img)
    pixels = np.array(Image.open(img_path))
    image.append(pixels)


In [ ]:
images = np.array(image)
print(type(images), images.shape)

In [ ]:
train_small = train[['file_name', 'bbox']].copy()
df_grouped = train_small.groupby('file_name', as_index=True).agg({'bbox': list})
bboxes = df_grouped['bbox'].values.tolist()

In [ ]:
import numpy as np

max_boxes = 100
padded_bboxes = np.zeros((len(bboxes), max_boxes, 4))
for i, boxes in enumerate(bboxes):
    padded_bboxes[i, :len(boxes), :] = boxes
padded_bboxes = padded_bboxes.reshape(len(bboxes), -1)  # Form: (n_samples, 4 * max_boxes)
padded_bboxes = padded_bboxes.tolist()


In [ ]:
from ultralytics import YOLO
model = YOLO('YOLO8n.pt')